# Mediana dla obrazu kolorowego

Idea filtracji medianowej jest dość prosta dla obrazów w odcieniach szarości.
Dla obrazów kolorowych trudniej jest określić kryterium wg. którego szeregowane będą wartości, z których wyznaczana będzie mediana.

Jedną z możliwości wykonania filtracji medianowej dla obrazów kolorowych (na podstawie *The Image Processing Handbook*, J. Russ) jest wykorzystanie następującej definicji mediany:
``mediana to ten piksel z otoczenia, którego odległość do innych pikseli z otoczenia jest najmniejsza''.
Jako miarę odległości wykorzystujemy pierwiastek z sumy kwadratów różnic poszczególnych składowych R,G,B.
Zatem odległość między dwoma pikselami wyraża się wzorem:
\begin{equation}
dRGB = \sqrt{(R_1-R_2)^2+(G_1-G_2)^2+(B_1-B_2)^2}
\end{equation}

Warto zwrócić uwagę, że istnieje wiele możliwości zdefiniowania porównywania wielkości wektorowych (jeden piksel to wektor o trzech składowych).
Można zamiast odległości wykorzystać kąt albo połączyć oba parametry.
Ponadto istnieje możliwość dodania do wektora dodatkowych składowych - tak aby lepiej opisać piksel.

Celem zadania jest implementacja opisanego algorytmu.

1. Wczytaj obraz *lenaRGBSzum.png* (dostępny na git).
2. Zdefiniuj rozmiar okna.
3. Wykonaj pętle po pikselach, dla których okno jest zdefiniowane (pomiń brzeg obrazu).
4. Dla każdego piksela pobierz okno o właściwym rozmiarze.
5. Wykonaj pętle po oknie, wewnątrz której obliczona zostanie suma odległości.
    - Obliczanie różnicy: `window - window[rowWin, colWin]`.
    - Obliczanie kwadratów: `np.square`.
    - Obliczanie pierwiastka: `np.sqrt`.
    - Obliczanie sumy metodą `.sum`.
6. Po obliczeniu macierzy odległości wyznacz argument elementu minimalnego.
Wykorzystaj funkcję `np.argmin`.
Argument funkcji zostanie spłaszczony, jeśli ma więcej niż jeden wymiar.
Aby przekonwertować spłaszczony indeks na indeks macierzy wykorzystaj funkcję `np.unravel_index`.
7. Przypisz odpowiedni wektor wartości do piksela obrazu wynikowego.
8. Wyświetl obraz oryginalny i przefiltrowany.
9. Przeprowadź dwa eksperymenty - dla obrazu _lenaRGB_ oraz _lenaRGBszum_.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/'

fileNames = ["lenaRGB.png", "lenaRGBSzum.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

In [ ]:
lena_rgb = cv2.imread('lenaRGB.png')
lena_rgb = cv2.cvtColor(lena_rgb, cv2.COLOR_BGR2RGB)

lena_rgb_noise = cv2.imread('lenaRGBSzum.png')
lena_rgb_noise = cv2.cvtColor(lena_rgb_noise, cv2.COLOR_BGR2RGB)

In [ ]:
def median_filter_color(image, window_size):
    # Pobranie wymiarów obrazu
    height, width, channels = image.shape
    
    # Inicjalizacja obrazu wynikowego
    filtered_image = np.copy(image)
    
    # Pętle po pikselach
    for i in range(window_size//2, height - window_size//2):
        for j in range(window_size//2, width - window_size//2):
            # Pobranie okna o właściwym rozmiarze
            window = image[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1]
            
            # Inicjalizacja list do przechowywania odległości
            distances = []
            
            # Pętle po oknie
            for k in range(window_size):
                for l in range(window_size):
                    # Obliczanie odległości między pikselami
                    diff = np.sqrt(np.sum(np.square(window[k, l] - window)))
                    distances.append(diff)
            
            # Wyznaczenie indeksu piksela o najmniejszej odległości
            min_index = np.argmin(distances)
            min_row, min_col = np.unravel_index(min_index, (window_size, window_size))
            
            # Przypisanie wartości piksela obrazu wynikowego
            filtered_image[i, j] = window[min_row, min_col]
    
    return filtered_image

In [ ]:
lena_rgb_median_3 = median_filter_color(lena_rgb, 3)
lena_rgb_median_5 = median_filter_color(lena_rgb, 5)

lena_noise_median_3 = median_filter_color(lena_rgb_noise, 3)
lena_noise_median_5 = median_filter_color(lena_rgb_noise, 5)

In [ ]:
plt.subplots(2, 3, figsize=(15,10))
plt.subplot(2, 3, 1)
plt.imshow(lena_rgb)
plt.title("Original")
plt.subplot(2, 3, 2)
plt.imshow(lena_rgb_median_3)
plt.title("Median 3x3")
plt.subplot(2, 3, 3)
plt.imshow(lena_rgb_median_5)
plt.title("Median 5x5")

plt.subplot(2, 3, 4)
plt.imshow(lena_rgb_noise)
plt.title("Original")
plt.subplot(2, 3, 5)
plt.imshow(lena_noise_median_3)
plt.title("Median 3x3")
plt.subplot(2, 3, 6)
plt.imshow(lena_noise_median_5)
plt.title("Median 5x5")

plt.show()